In [ ]:
#Importing required packages
#Make sure you have installed all the packages below
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rc
rc('text', usetex=True)
from tqdm import tqdm

#### SM Parameters

In [ ]:
#Defining fundamental constants to be used later
m_kaon, m_muon, m_electron, m_pion = 0.493677, 0.105658, 0.000511, 0.13957 #particle masses in GeV
br_k2_mu_sm = 0.6356 #Branching ratio for K+ -> mu+ nu in SM
x0 = 2.0e-3 #Prefactor for decay of K+ -> pi+ S

GF = 1.16e-5 #Fermi constant in GeV^-2
vev = 246.0 #Higgs vev in GeV
sw2 = 0.223 #Weak mixing angle squared

# Part 1: Limit Plot Comparison

#### Defining the branching ratio of a kaon into a charged lepton and a heavy neutral lepton $N$
##### Eq. (2) and following of [https://arxiv.org/abs/2106.06548]

In [ ]:
def rho_N(x, y):
    return (x + y - (x-y)**2)*np.sqrt(1 + x**2 + y**2 - 2*(x+y+x*y))/(x*(1-x)**2)

def branching_k_to_ell_N(br_k2_sm, m_ell, m_N, mK, usq):
    return br_k2_sm*usq*rho_N(m_ell**2/mK**2, m_N**2/mK**2)

#### Defining the branching ratio of a kaon into a charged pion and a Higgs-portal scalar $S$

In [ ]:
def rho_S(x, y):
    return 0.5*np.sqrt(1 + x**2 + y**2 - 2*(x + y +x*y))

def branching_k_to_pi_phi(x0, m_pi, m_S, mK, ssq_theta):
    return x0*ssq_theta*rho_S(m_S**2/mK**2, m_pi**2/mK**2)

#### Plot the branching ratio of a kaon into a muon and an HNL as a function of HNL mass

In [ ]:
m_n_array = np.logspace(-3, np.log10(0.99*(m_kaon-m_muon)), 101)
u_sq_test = 1.0e-4
plt.plot(m_n_array, branching_k_to_ell_N(br_k2_mu_sm, m_muon, m_n_array, m_kaon, u_sq_test), lw=2)
plt.gca().set_xscale('log')
plt.gca().set_yscale('log')
plt.gca().set_ylim([1e-5, 1e-3])

plt.gca().set_xlabel(r"$m_N\ [\mathrm{GeV}]$", fontsize=16)
plt.gca().set_ylabel(r"$\mathrm{Br}\left(K\to \mu N\right)$", fontsize=16)

#### Functions to define the signal width of the HNL

In [ ]:
def LL(yl):
    return -4*np.log((1 + np.sqrt(1 - 4*yl**2))/(2*yl))

def signal_width_N(m_lep, mN, Usq, GF, sw2):
    prefactor = GF**2 * mN**5/(192*np.pi**3)*Usq
    yl = m_lep/mN

    term1 = 0.25*(1 - 4*sw2 + 8*sw2**2)*((1 - 14*yl**2 - 2*yl**4 - 12*yl**6)*np.sqrt(1 - 4*yl**2) + 12*yl**4*(yl**4 - 1)*LL(yl))
    term2 = 4.0*(0.5*sw2*(2*sw2-1))*(yl**2*(2+10*yl**2-12*yl**4)*np.sqrt(1 - 4*yl**2) + 6*yl**4*(1-2*yl**2 + 2*yl**4)*LL(yl))

    return 2*prefactor*(term1 + term2) #Factor of two: Majorana HNL decay instead of Dirac

#### Function to define the signal width of the HPS

In [ ]:
def signal_width_S(m_lep, m_S, ssqth, v):
    return ssqth*m_lep**2*m_S/(8*np.pi*v**2)*(1 - 4*m_lep**2/m_S**2)**(1.5)

def Enew_twobodydecay(m_parent, m_sm, m_new):
    """Energy in the parent decay rest-frame of an outgoing particle "new" 
    in a two-body decay parent -> sm + new
    """
    return (m_parent**2 - m_sm**2 + m_new**2)/(2*m_parent)

In [ ]:
fig, ax1 = plt.subplots()

m_n_array_linear = np.linspace(0, 0.2, 101)
ax1.plot(m_n_array_linear, branching_k_to_ell_N(br_k2_mu_sm, m_muon, m_n_array_linear, m_kaon, 1.0e-4)/branching_k_to_pi_phi(x0, m_pion, m_n_array_linear, m_kaon, 1.0e-6), lw=2, color='C0')
ax1.set_yscale('log')
ax1.set_xlim([0, 0.2])
ax1.set_ylim([1e4, 1e6])
ax1.set_xlabel(r"$m_S = m_N\ [\mathrm{GeV}]$", fontsize=16)
ax1.set_ylabel(r"$\Phi_{N}/\Phi_{S}$", fontsize=16, color='C0')
ax1.tick_params(axis='y', labelcolor='C0')

ax2 = ax1.twinx()

m_n_array_linear = np.linspace(0.0011, 0.2, 101)

ratio_of_decay_probabilities = (signal_width_N(m_electron, m_n_array_linear, 1.0e-4, GF, sw2))*Enew_twobodydecay(m_kaon, m_pion, m_n_array_linear)/(signal_width_S(m_electron, m_n_array_linear, 1.0e-6, vev)*Enew_twobodydecay(m_kaon, m_muon, m_n_array_linear))

ax2.plot(m_n_array_linear, ratio_of_decay_probabilities, lw=2, color='red')
#make label on y-axis for ax2 on the right
ax2.set_ylabel(r"$P_{\mathrm{decay,\ }N}/P_{\mathrm{decay,\ }S}$", fontsize=16, color='red', rotation=270)
ax2.yaxis.set_label_coords(1.15, 0.5)
ax2.tick_params(axis='y', labelcolor='red')
ax2.set_yscale('log')
ax2.set_xlim([0, 0.2])
ax2.set_ylim([1e-12, 1e-2])


In [ ]:
#Scalar masses, limits on \theta, and efficiency provided by MicroBooNE
scalar_masses = np.array([0.001023, 0.003, 0.009, 0.027, 0.054, 0.081, 0.1, 0.11, 0.12, 0.13, \
0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2, 0.21])
theta_limits = np.array([0.18, 0.0069, 0.0038, 0.0018, 0.00096, 0.00058, 0.00046, 0.00043, \
0.0004, 0.00038, 0.00037, 0.00035, 0.00034, 0.00033, 0.00033, \
0.00033, 0.00033, 0.00035])
microboone_efficiencies = np.array([0.003, 0.004, 0.004, 0.009, 0.028, 0.085, 0.112, 0.131, 0.140, 0.146, \
0.170, 0.151, 0.149, 0.144, 0.133, 0.118, 0.098, 0.096])

In [ ]:
def signal_ratio(m_N, m_S, usq, ssq_theta):
   production_ratio = branching_k_to_ell_N(br_k2_mu_sm, m_muon, m_N, m_kaon, usq)/branching_k_to_pi_phi(x0, m_pion, m_S, m_kaon, ssq_theta)
   ratio_of_decay_probabilities = (signal_width_N(m_electron, m_N, usq, GF, sw2))*Enew_twobodydecay(m_kaon, m_pion, m_S)/(signal_width_S(m_electron, m_S, ssq_theta, vev)*Enew_twobodydecay(m_kaon, m_muon, m_N))

   return production_ratio * ratio_of_decay_probabilities


#### Determine the value of $|U|^2$ for which this ratio is 1 for a given $m_N = m_S = 100$ MeV

In [ ]:
m_S_test = scalar_masses[6]
ssq_theta_limit_test = np.sin(theta_limits[6])**2
m_N_test = m_S_test

usq_test = np.logspace(-7, -2, 101)

signal_ratio_test = signal_ratio(m_N_test, m_S_test, usq_test, ssq_theta_limit_test)

In [ ]:
fig, ax = plt.subplots()

ax.plot(usq_test, signal_ratio_test, lw=2, color='dodgerblue', label=r'$m_S = m_N = 100\ \mathrm{MeV}$', zorder=20)
ax.set_xscale('log')
ax.set_yscale('log')
ax.axis([1e-7, 1e-2, 1e-4, 1e4])
ax.legend(loc=4, fontsize=16, framealpha=1)

ax.axhline(1.0, lw=1, ls='--', color='k')
ax.axvline(1/np.sqrt(signal_ratio(m_N_test, m_S_test, 1.0, ssq_theta_limit_test)), lw=1, ls='--', color='k')

ax.set_xlabel(r"$|U_{\mu N}|^2$", fontsize=16)
ax.set_ylabel(r"$R_{N}/R_{S} \propto |U_{\mu N}|^4$", fontsize=16)

In [ ]:
usq_limits = 1/np.sqrt(signal_ratio(scalar_masses, scalar_masses, 1.0, np.sin(theta_limits)**2))

In [ ]:
PIENU = np.loadtxt("./PIENU_mu_data.csv", delimiter=",")
PS191 = np.loadtxt("./PS191_mu_data.csv", delimiter=",")
Michel = np.loadtxt("./MichelSpectrum.csv", delimiter=",")
KEK = np.loadtxt("./KEK_data.csv", delimiter=",")
E949 = np.loadtxt("./E949_data.csv", delimiter=",")

In [ ]:
colpal = sns.color_palette("Paired", 10)

redcol = colpal[5]
grncol = colpal[3]
bluecol = colpal[1]
nexcol = colpal[7]
newercol = colpal[9]

bluecolfill=(0.6509803921568628, 0.807843137254902, 0.8901960784313725, 0.5)
redcolfill=(0.984313725490196, 0.6039215686274509, 0.6, 0.5)
grncolfill=(0.698039, 0.8745098, 0.5411765, 0.5)
newerfill=(0.792156862745098, 0.6980392156862745, 0.8392156862745098, 0.5)
nexcolfill=(0.9921568627450981, 0.7490196078431373, 0.43529411764705883, 0.5)


In [ ]:
fig, ax = plt.subplots()

ax.plot(scalar_masses, usq_limits, lw=2, color='k', label=r'$\varepsilon(m_N) = \varepsilon(m_S)$')
ax.set_xscale('log')
ax.set_yscale('log')
ax.axis([0.020, 0.200, 1e-7, 1e-2])
ax.legend(loc=3, fontsize=16)

ax.fill_between(10**Michel.T[0], [1.0 for k in range(len(Michel))], 10**Michel.T[1], lw=2, edgecolor=redcol, facecolor=redcolfill, interpolate=True)
ax.fill_between(10**KEK.T[0], [1.0 for k in range(len(KEK))], 10**KEK.T[1], lw=2, edgecolor=grncol, facecolor=grncolfill, interpolate=True)
ax.fill_between(10**PS191.T[0], [1.0 for k in range(len(PS191))], 10**PS191.T[1], lw=2, edgecolor=bluecol, facecolor=bluecolfill, interpolate=True)
ax.fill_between(10**E949.T[0], [1.0 for k in range(len(E949))], 10**E949.T[1], lw=2, edgecolor=nexcol, facecolor=nexcolfill, interpolate=True)
ax.fill_between(10**PIENU.T[0], [1.0 for k in range(len(PIENU))], 10**PIENU.T[1], lw=2, edgecolor=newercol, facecolor=newerfill, interpolate=True)

a1 = r"$\mathrm{PIENU}$"
ax.annotate(a1, xy=(0.025, 5e-5), xytext=(0, 0), textcoords="offset points", ha="center", va="center", size=14, zorder=20, color=newercol)

a2 = r"$\mathrm{Michel}$"
ax.annotate(a2, xy=(0.043, 2e-3), xytext=(0, 15), textcoords="offset points", ha="center", va="center", size=14, zorder=20, color=redcol)
a2b = r"$\mathrm{electron}$"
ax.annotate(a2b, xy=(0.043, 2e-3), xytext=(0, 0), textcoords="offset points", ha="center", va="center", size=14, zorder=20, color=redcol)

a3 = r"$\mathrm{KEK}$"
ax.annotate(a3, xy=(0.081, 1e-4), xytext=(0, 0), textcoords="offset points", ha="center", va="center", size=14, zorder=20, color=grncol)

a4 = r"$\mathrm{PS191}$"
ax.annotate(a4, xy=(0.150, 2e-4), xytext=(0, 0), textcoords="offset points", ha="center", va="center", size=14, zorder=20, color=bluecol)

a5 = r"$\mathrm{E949}$"
ax.annotate(a5, xy=(0.190, 1e-5), xytext=(0, -25), textcoords="offset points", ha="center", va="center", size=14, zorder=20, color=nexcol, rotation=90)

ax.set_xlabel(r"$m_N\ \mathrm{[GeV]}$", fontsize=20)
ax.set_ylabel(r"$\left| U_{\mu 4}\right|^2$", fontsize=20)

# Part 2: Decay Kinematics

### Perform an analysis on HPS and HNL decays into electron/positron pairs. How do the event kinematics look in MicroBooNE?

In [ ]:
import HNLGen, LabFrame
import matplotlib as mpl

In [ ]:
detector_angular_uncertainty = 3.0 #degrees

### Create several distributions: HNL with $m_N = 100$ MeV

In [ ]:
gLgRTrue = [0.5*(1.0 - 2.0*sw2), sw2]

mN_test = 0.100

distribution_100MeV_HNL = HNLGen.RetSampDM([mN_test, m_electron, m_electron], [m_kaon, m_muon], gLgRTrue, 1, False, False) #Generate sample of events
labframe_dist_100MeV_HNL = LabFrame.LFEvts(distribution_100MeV_HNL, [mN_test, m_electron, m_electron], [m_kaon, m_muon])
labframe_smeardist_100MeV_HNL = LabFrame.LFSmear(labframe_dist_100MeV_HNL, detector_angular_uncertainty)

event_kinematics_truth_100MeV_HNL = LabFrame.LFAnalysis(labframe_dist_100MeV_HNL)
event_kinematics_reco_100MeV_HNL = LabFrame.LFAnalysis(labframe_smeardist_100MeV_HNL)

### Create several distributions: HPS with $m_S = 100$ MeV and $80$ MeV

In [ ]:
mS_test = 0.100
labframe_dist_100MeV_HPS = LabFrame.LFEvtsHPS([mS_test, m_electron], [m_kaon, m_pion], int(1e6))
labframe_smeardist_100MeV_HPS = LabFrame.LFSmear(labframe_dist_100MeV_HPS, detector_angular_uncertainty)

event_kinematics_truth_100MeV_HPS = LabFrame.LFAnalysis(labframe_dist_100MeV_HPS)
event_kinematics_reco_100MeV_HPS = LabFrame.LFAnalysis(labframe_smeardist_100MeV_HPS)


mS_test_2 = 0.080
labframe_dist_80MeV_HPS = LabFrame.LFEvtsHPS([mS_test_2, m_electron], [m_kaon, m_pion], int(1e6))
labframe_smeardist_80MeV_HPS = LabFrame.LFSmear(labframe_dist_80MeV_HPS, detector_angular_uncertainty)

event_kinematics_truth_80MeV_HPS = LabFrame.LFAnalysis(labframe_dist_80MeV_HPS)
event_kinematics_reco_80MeV_HPS = LabFrame.LFAnalysis(labframe_smeardist_80MeV_HPS)

### Perform analysis/reconstruction on each sample -- truth and reco. information

In [ ]:
minimum_visible_energy = 0.010 #10 MeV -- minimum visible energy
minimum_openingangle = 10.0 #degrees -- minimum opening angle of a pair of electrons

result_100MeV_truth_HNL = np.array(LabFrame.CutAnalysis(event_kinematics_truth_100MeV_HNL, minimum_visible_energy, minimum_openingangle, VB=False)[1])
result_100MeV_reco_HNL = np.array(LabFrame.CutAnalysis(event_kinematics_reco_100MeV_HNL, minimum_visible_energy, minimum_openingangle, VB=False)[1])

result_100MeV_truth_HPS = np.array(LabFrame.CutAnalysis(event_kinematics_truth_100MeV_HPS, minimum_visible_energy, minimum_openingangle, VB=False)[1])
result_100MeV_reco_HPS = np.array(LabFrame.CutAnalysis(event_kinematics_reco_100MeV_HPS, minimum_visible_energy, minimum_openingangle, VB=False)[1])

result_80MeV_truth_HPS = np.array(LabFrame.CutAnalysis(event_kinematics_truth_80MeV_HPS, minimum_visible_energy, minimum_openingangle, VB=False)[1])
result_80MeV_reco_HPS = np.array(LabFrame.CutAnalysis(event_kinematics_reco_80MeV_HPS, minimum_visible_energy, minimum_openingangle, VB=False)[1])

### Plot distributions -- opening angle of $e^+ e^-$ pair vs. direction of leading charged lepton

In [ ]:
figwid = 4.0
fighei = 4.0
lside = 1.1
rside = 0.1
wwspace = 1.5

ncol = 1
nrow = 1

wid = lside + ncol*figwid + (ncol-1)*wwspace + rside

bot = 0.9
top = 0.1
hhspace = 1.25

hei = bot + nrow*fighei + (nrow-1)*hhspace + top

lfactor = lside/wid
rfactor = rside/wid
bfactor = bot/hei
tfactor = top/hei
wfactor = wwspace/figwid
hfactor = hhspace/fighei

fig, axes = plt.subplots(nrow, ncol, figsize=(wid, hei), facecolor='1.0');
fig.subplots_adjust(left = lfactor, bottom=bfactor, right=(1.0-rfactor), top=(1.0-tfactor), wspace=wfactor, hspace=hfactor);

ax = axes
xmin = -1
xmax = 1
ymin = 0
ymax = 30
ax.axis([xmin, xmax, ymin, ymax])
ax.set_xlabel(r'$\cos\theta_{e^+e^-}$', fontsize=16)        
ax.set_ylabel(r'$\theta_{e_{\rm lead}}\ [\mathrm{deg.}]$', fontsize=16)
            
ax.tick_params(direction='in', reset=True, which='both', zorder=30)
[l.set_position((0.5, -0.015)) for l in ax.get_xticklabels()]
[l.set_size(12) for l in ax.get_xticklabels()]
[l.set_size(12) for l in ax.get_yticklabels()]

ax.hist2d(result_100MeV_truth_HNL.T[2], np.arctan(result_100MeV_truth_HNL.T[3])*180.0/np.pi, bins = [np.linspace(-1, 1, 41), np.linspace(0, 30, 41)], weights=result_100MeV_truth_HNL.T[4], cmin=3e-9)#, cmin=1e-9, edgecolors='face')
ax.hist2d(result_100MeV_truth_HPS.T[2], np.arctan(result_100MeV_truth_HPS.T[3])*180.0/np.pi, bins = [np.linspace(-1, 1, 61), np.linspace(0, 30, 61)], weights=result_100MeV_truth_HPS.T[4], cmap='inferno', cmin=1e-5)#, cmin=1e-9, edgecolors='face')
ax.hist2d(result_80MeV_truth_HPS.T[2], np.arctan(result_80MeV_truth_HPS.T[3])*180.0/np.pi, bins = [np.linspace(-1, 1, 61), np.linspace(0, 30, 61)], weights=result_80MeV_truth_HPS.T[4], cmap='plasma', cmin=1e-5)#, cmin=1e-9, edgecolors='face')

ax.set_xlim([xmin,xmax])
ax.set_ylim([ymin,ymax])

ax.annotate(r"$\mathrm{HPS},\ m_S = 100\ \mathrm{MeV}$", xy=(0.2, 7), xytext=(0,15), textcoords="offset points", ha="right", va="bottom", size=16, color=(mpl.cm.get_cmap('inferno'))(0.8))
ax.annotate(r"$m_S = 80\ \mathrm{MeV}$", xy=(0, 2), xytext=(-35,0), textcoords="offset points", ha="left", va="top", size=16, color=(mpl.cm.get_cmap('plasma'))(0.8))
ax.annotate(r"$\mathrm{HNL},\ m_N = 100\ \mathrm{MeV}$", xy=(0.4, 25), xytext=(22,0), textcoords="offset points", ha="right", va="top", size=16, color=(mpl.cm.get_cmap('viridis'))(0.8))

### Inspect efficiencies provided by MicroBooNE

In [ ]:
plt.plot(scalar_masses, microboone_efficiencies, lw=2, color='k', ls='-')
plt.gca().set_xlabel(r"$m_S\ [\mathrm{GeV}]$", fontsize=16)
plt.gca().set_ylabel(r"$\varepsilon(m_S)$", fontsize=16)

### How does this relate to opening angle of scalar decay-product $e^+ e^-$ pairs? Seems to peak around 140 MeV

In [ ]:
NS = int(1e6)
scalar_mean_opening_angle, scalar_cut_efficiency = [], []
for scalar_mass_test in tqdm(scalar_masses):
    lab_frame_events = LabFrame.LFEvtsHPS([scalar_mass_test, m_electron], [m_kaon, m_pion], NS)
    lab_frame_smeared = LabFrame.LFSmear(lab_frame_events, detector_angular_uncertainty)

    analyzed_events = LabFrame.LFAnalysis(lab_frame_smeared)
    results_cut = LabFrame.CutAnalysis(analyzed_events, minimum_visible_energy, minimum_openingangle)

    cut_efficiency = results_cut[0]
    scalar_cut_efficiency.append([scalar_mass_test, cut_efficiency])

    events_pass_cuts = results_cut[1]
    if events_pass_cuts is not None:
        mean_opening_angle = np.average(np.transpose(events_pass_cuts)[2], weights=np.transpose(events_pass_cuts)[4])
        scalar_mean_opening_angle.append([scalar_mass_test, mean_opening_angle])

scalar_mean_opening_angle = np.array(scalar_mean_opening_angle)
scalar_cut_efficiency = np.array(scalar_cut_efficiency)

### Scalar mass vs. Average opening Angle

In [ ]:
plt.plot(scalar_masses, scalar_mean_opening_angle.T[1], lw=2, color='k', ls='-')
plt.gca().set_xlabel(r'$m_S\ \mathrm{[GeV]}$', fontsize=16)        
plt.gca().set_ylabel(r'$\overline{\cos(\theta_{e^+ e^-})}$', fontsize=16)
plt.gca().set_ylim(-1,1)

### Average opening Angle vs. Efficiency

In [ ]:
plt.plot(scalar_mean_opening_angle.T[1], microboone_efficiencies, lw=2, color='k', ls='-')
plt.gca().set_ylabel(r'$\varepsilon(m_S)$', fontsize=16)
plt.gca().set_xlabel(r'$\overline{\cos(\theta_{e^+ e^-})}$', fontsize=16)
plt.gca().set_xlim(-1,1)

### Assumption: Efficiency depends on mean opening angle.
### Determine mean opening angle for HNLs with similar masses, then determine the efficiency based on this assumption

In [ ]:
hnl_mean_opening_angle, hnl_cut_efficiency = [], []
for hnl_mass_test in tqdm(scalar_masses):
    initial_distribution = HNLGen.RetSampDM([hnl_mass_test, m_electron, m_electron], [m_kaon, m_muon], gLgRTrue, 1, False, True)
    lab_frame_events = LabFrame.LFEvts(initial_distribution, [hnl_mass_test, m_electron, m_electron], [m_kaon, m_muon])
    lab_frame_smeared = LabFrame.LFSmear(lab_frame_events, detector_angular_uncertainty)

    analyzed_events = LabFrame.LFAnalysis(lab_frame_smeared)
    results_cut = LabFrame.CutAnalysis(analyzed_events, minimum_visible_energy, minimum_openingangle)

    cut_efficiency = results_cut[0]
    hnl_cut_efficiency.append([hnl_mass_test, cut_efficiency])

    events_pass_cuts = results_cut[1]
    if events_pass_cuts is not None:
        mean_opening_angle = np.average(np.transpose(events_pass_cuts)[2], weights=np.transpose(events_pass_cuts)[4])
        hnl_mean_opening_angle.append([hnl_mass_test, mean_opening_angle])

hnl_mean_opening_angle = np.array(hnl_mean_opening_angle)
hnl_cut_efficiency = np.array(hnl_cut_efficiency)

In [ ]:
plt.plot(scalar_masses, scalar_mean_opening_angle.T[1], lw=2, color='k', ls='-')
plt.plot(scalar_masses, hnl_mean_opening_angle.T[1], lw=2, color='k', ls='--')
plt.gca().set_xlabel(r'$m_S\ \mathrm{[GeV]}$', fontsize=16)        
plt.gca().set_ylabel(r'$\overline{\cos(\theta_{e^+ e^-})}$', fontsize=16)
plt.gca().set_ylim(-1,1)

In [ ]:
from scipy.interpolate import interp1d

#Determine the HNL efficiency by interpolation
efficiency_vs_scalar_opening_angle = interp1d(scalar_mean_opening_angle.T[1], microboone_efficiencies, fill_value=0.0, bounds_error=False)
hnl_efficiency = efficiency_vs_scalar_opening_angle(hnl_mean_opening_angle.T[1])

### Plot HNL Efficiencies against Scalar One

In [ ]:
xmin, xmax, ymin, ymax = 0, 0.2, 0, 0.2

plt.plot(scalar_masses, microboone_efficiencies, lw=2, color='k', ls='-')
plt.plot(scalar_masses, hnl_efficiency, lw=2, color='k', ls='--')

#Most conservative option: incorporate cuts on minimum energy/minimum opening angle in efficiency too
plt.plot(scalar_masses, hnl_efficiency*hnl_cut_efficiency.T[1]/scalar_cut_efficiency.T[1], lw=2, color='k', ls='-.')

plt.gca().set_xlabel(r'$m_S\ \mathrm{[GeV]}$', fontsize=16)        
plt.gca().set_ylabel(r'$\varepsilon(m_S)$', fontsize=16)
plt.gca().axis([xmin, xmax, ymin, ymax])


### Determine the most conservative event-reweighting factor based on the different approaches

In [ ]:
conservative_efficiency_factor = []
for i in range(len(scalar_masses)):
    scalar_e = microboone_efficiencies[i]
    hnl_e = hnl_efficiency[i]

    scalar_cut_impact = scalar_cut_efficiency[i][1]
    hnl_cut_impact = hnl_cut_efficiency[i][1]

    reweight_factor = np.min([1.0, hnl_e/scalar_e, hnl_e/scalar_e*hnl_cut_impact/scalar_cut_impact])
    conservative_efficiency_factor.append(reweight_factor)
conservative_efficiency_factor = np.array(conservative_efficiency_factor)

### Determine updated limits with event rate rescaled by this efficiency factor

In [ ]:
usq_limits_conservative = usq_limits/np.sqrt(conservative_efficiency_factor)

In [ ]:
fig, ax = plt.subplots()

ax.plot(scalar_masses, usq_limits, lw=2, color='k', label=r'$\varepsilon(m_N) = \varepsilon(m_S)$')
ax.plot(scalar_masses, usq_limits_conservative, lw=2, color='k', ls='--', label=r'$\varepsilon(m_N) = \varepsilon(\overline{\cos\theta_{e^+ e^-}})$')
ax.set_xscale('log')
ax.set_yscale('log')
ax.axis([0.020, 0.200, 1e-7, 1e-2])
ax.legend(loc=3, fontsize=16)

ax.fill_between(10**Michel.T[0], [1.0 for k in range(len(Michel))], 10**Michel.T[1], lw=2, edgecolor=redcol, facecolor=redcolfill, interpolate=True)
ax.fill_between(10**KEK.T[0], [1.0 for k in range(len(KEK))], 10**KEK.T[1], lw=2, edgecolor=grncol, facecolor=grncolfill, interpolate=True)
ax.fill_between(10**PS191.T[0], [1.0 for k in range(len(PS191))], 10**PS191.T[1], lw=2, edgecolor=bluecol, facecolor=bluecolfill, interpolate=True)
ax.fill_between(10**E949.T[0], [1.0 for k in range(len(E949))], 10**E949.T[1], lw=2, edgecolor=nexcol, facecolor=nexcolfill, interpolate=True)
ax.fill_between(10**PIENU.T[0], [1.0 for k in range(len(PIENU))], 10**PIENU.T[1], lw=2, edgecolor=newercol, facecolor=newerfill, interpolate=True)

a1 = r"$\mathrm{PIENU}$"
ax.annotate(a1, xy=(0.025, 5e-5), xytext=(0, 0), textcoords="offset points", ha="center", va="center", size=14, zorder=20, color=newercol)

a2 = r"$\mathrm{Michel}$"
ax.annotate(a2, xy=(0.043, 2e-3), xytext=(0, 15), textcoords="offset points", ha="center", va="center", size=14, zorder=20, color=redcol)
a2b = r"$\mathrm{electron}$"
ax.annotate(a2b, xy=(0.043, 2e-3), xytext=(0, 0), textcoords="offset points", ha="center", va="center", size=14, zorder=20, color=redcol)

a3 = r"$\mathrm{KEK}$"
ax.annotate(a3, xy=(0.081, 1e-4), xytext=(0, 0), textcoords="offset points", ha="center", va="center", size=14, zorder=20, color=grncol)

a4 = r"$\mathrm{PS191}$"
ax.annotate(a4, xy=(0.150, 2e-4), xytext=(0, 0), textcoords="offset points", ha="center", va="center", size=14, zorder=20, color=bluecol)

a5 = r"$\mathrm{E949}$"
ax.annotate(a5, xy=(0.190, 1e-5), xytext=(0, -25), textcoords="offset points", ha="center", va="center", size=14, zorder=20, color=nexcol, rotation=90)

ax.set_xlabel(r"$m_N\ \mathrm{[GeV]}$", fontsize=20)
ax.set_ylabel(r"$\left| U_{\mu 4}\right|^2$", fontsize=20)